In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional

In [ ]:
class WorkOrderSearch(BaseModel):
    issue: str = Field(
        ...,
        description="Issue reported by the maintenance tech to search work orders. For example, air entrapment or incomplete curing",
    )
    temporal: str = Field(
        ...,
        description="Temporal string that refers to the first or last `n` work orders. Options can be 'first', 'last' or 'all'",
    )
    n: Optional[int] = Field(..., description="Number of work orders requested")


@tool(args_schema=WorkOrderSearch)
def search_work_orders(issue: str, temporal: str, n: Optional[int]) -> List:
    """Search for work orders where a similar issue was observed and/or resolved."""
    print({"issue": issue, "temporal": temporal, "n": n})

    return [
        {
            "id": "12345",
            "issue": "Uneven pressure distribution",
            "solution": "Ensure the mold and platens are clean and properly aligned. Regularly calibrate the machine to maintain even pressure. Check for and replace any worn or damaged seals.",
        },
        {
            "id": "12346",
            "issue": "Incomplete curing",
            "solution": "Verify that the heating elements are functioning correctly and uniformly. Adjust the curing time and temperature settings according to material specifications. Regularly inspect and maintain heating elements and thermocouples.",
        },
        {
            "id": "12347",
            "issue": "Material flash",
            "solution": "Check mold clamping force and ensure it is adequate. Inspect the mold for wear and tear and repair or replace if necessary. Ensure proper material loading to prevent excess material overflow.",
        },
        {
            "id": "12348",
            "issue": "Air entrapment",
            "solution": "Implement proper venting techniques in the mold design. Use vacuum-assisted molding if necessary. Ensure the material is loaded correctly to minimize air pockets.",
        },
        {
            "id": "12349",
            "issue": "Sticking of parts to mold",
            "solution": "Apply an appropriate mold release agent before each cycle. Inspect and clean the mold surfaces regularly. Consider using molds with better release properties or modifying the mold design to reduce sticking.",
        },
    ][:n]

In [ ]:
class PartsSearch(BaseModel):
    partName: Optional[str] = (Field(..., description="Name or description of the part"),)
    partNumber: Optional[str] = Field(..., description="Part number or ID")


@tool(args_schema=PartsSearch)
def search_parts(partName: Optional[str], partNumber: Optional[str]) -> List:
    """Search for replacement parts"""
    print({"partName": partName, "partNumber": partNumber})

    return [
        {
            "partName": "Heating Element",
            "partNumber": "HE-12345",
            "inventory": 15,
            "location": "Aisle 3, Shelf B",
        },
        {
            "partName": "Platen",
            "partNumber": "PL-67890",
            "inventory": 8,
            "location": "Aisle 1, Shelf D",
        },
        {
            "partName": "Hydraulic Cylinder",
            "partNumber": "HC-11223",
            "inventory": 5,
            "location": "Aisle 2, Shelf A",
        },
        {
            "partName": "Control Panel",
            "partNumber": "CP-44556",
            "inventory": 3,
            "location": "Aisle 4, Shelf C",
        },
        {
            "partName": "Thermocouple",
            "partNumber": "TC-78901",
            "inventory": 20,
            "location": "Aisle 3, Shelf F",
        },
    ]

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain_openai import AzureChatOpenAI

tools = [search_parts, search_work_orders]
functions = [convert_to_openai_function(f) for f in tools]
model = AzureChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    api_version="2024-02-01",
    azure_deployment=os.environ["OPENAI_API_DEPLOYMENT"],
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
).bind(functions=functions)
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful but sassy assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)

qa = AgentExecutor(agent=chain, tools=tools, verbose=True, memory=memory)

In [ ]:
# qa.invoke({"input": "What are the first 2 work orders that address parts sticking to mold?"})

qa.invoke({"input": "I need to replace a Hydraulic Cylinder. Where would I find one, and what is the current inventory?"})